In [1]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')

Overwriting utility.py


In [2]:
import pandas as pd
df = pd.read_csv("france.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27381995 entries, 0 to 27381994
Data columns (total 11 columns):
 #   Column    Dtype  
---  ------    -----  
 0   LON       float64
 1   LAT       float64
 2   NUMBER    float64
 3   STREET    object 
 4   UNIT      float64
 5   CITY      object 
 6   DISTRICT  float64
 7   REGION    float64
 8   POSTCODE  float64
 9   ID        float64
 10  HASH      object 
dtypes: float64(8), object(3)
memory usage: 2.2+ GB


In [8]:
%%writefile file.yaml
file_type: csv
dataset_name: france
file_name: Parking_Violations_2016
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - lon
    - lat
    - number
    - street 
    - unit
    - city
    - district
    - region
    - postcode
    - ID
    - HASH

Overwriting file.yaml


In [4]:
import utility
config_data = utility.read_config_file("file.yaml")
config_data

{'file_type': 'csv',
 'dataset_name': 'YellowTripData',
 'file_name': 'Parking_Violations_2016',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['lon',
  'lat',
  'number',
  'street',
  'unit',
  'city',
  'district',
  'region',
  'postcode',
  'ID',
  'HASH']}

In [5]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['LON', 'LAT', 'NUMBER', 'STREET', 'UNIT', 'CITY', 'DISTRICT', 'REGION',
       'POSTCODE', 'ID', 'HASH'],
      dtype='object')
columns of YAML are: ['lon', 'lat', 'number', 'street', 'unit', 'city', 'district', 'region', 'postcode', 'ID', 'HASH']


In [6]:
utility.col_header_val(df,config_data)

column name and column length validation passed


In [7]:
df.to_csv('france.gz', compression = 'gzip', sep = '|', header = False, index = False)